# Лабораторная работа 6. 
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы БВТ2002 Айвазян Арина

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом. 

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса. 

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена



### Выполнение:

In [ ]:
import math
import time
import networkx as nx
import ipywidgets as widgets
import pandas as pd
from collections import deque

#### Алгоритм Флойда-Уоршелла

In [ ]:
def floyd_warshall(G, start, end):
    start -= 1
    end -= 1
    W = make_adjacency_matrix(G)
    for i in range(len(W)):
        for j in range(len(W[i])):
            if W[i][j] == 0 and i != j:
                W[i][j] = math.inf
    n = len(W)
    A = [[W[i][j] for i in range(n)] for j in range(n)] 
    V = [[math.inf for i in range(n)] for j in range(n)]
    for k in range(n): 
        for i in range(n):
            for j in range(n): 
                if A[i][j] > (A[i][k] + A[k][j]):
                    A[i][j] = A[i][k] + A[k][j]
                    V[i][j] = k+1
    path = [end+1, V[start][end]]
    if path[-1] != math.inf:
        while end != start+1:
            end = V[start][end]-1
            path.append(end)
    else:
        path.pop()
        path.append(start+1)
    return path[::-1]

In [ ]:
start_time = time.time()
G = make_graph(read_file("adjacency_matrix.txt"), 1)

res = floyd_warshall(G, 1, 6)
print("Кратчайший путь:", res)
print("time: ","%s seconds" % (time.time() - start_time))

new_graph(G, res).show("basic.html")

#### Алгоритм Дейкстры

In [ ]:
def dijkstra(G, start, end):
    start -= 1
    end -= 1
    w = make_adjacency_matrix(G)
    for i in range(len(w)):
        for j in range(len(w[i])):
            if w[i][j] == 0 and i != j:
                w[i][j] = math.inf
    n = len(w)
    dist = [math.inf] * n
    dist[start] = 0
    prev = [None] * n
    used = [False] * n
    min_dist = 0
    min_vertex = start
    while min_dist < math.inf:
        i = min_vertex 
        used[i] = True 
        for j in range(n): 
            if dist[i] + w[j][i] < dist[j]: 
                dist[j] = dist[i] + w[j][i] 
                prev[j] = i+1
        min_dist = math.inf
        for j in range(n):
            if not used[j] and dist[j] < min_dist:
                min_dist = dist[j]
                min_vertex = j
    path = [end+1]
    while end != start:
        end = prev[end]-1
        path.append(end+1)
    return path[::-1]

In [ ]:
start_time = time.time()
G = make_graph(read_file("adjacency_matrix.txt"), 1)
res = dijkstra(G, 1, 6)
print("Кратчайший путь:", res)
print("time: ","%s seconds" % (time.time() - start_time))

new_graph(G, res).show("basic.html")

#### Алгоритм Беллмана-Форда

In [ ]:
def bellman_ford(G, start, end):
    start -= 1
    end -= 1
    W = make_adjacency_matrix(G)
    for i in range(len(W)):
        for j in range(len(W[i])):
            if W[i][j] == 0 and i != j:
                W[i][j] = math.inf
    N = len(W)
    F = [[math.inf] * N for i in range(N)]
    V = [[math.inf] * N for i in range(N)]
    F[0][start] = 0 
    
    for k in range(1, N): 
        for i in range(N): 
            F[k][i] = F[k - 1][i]
            for j in range(N):
                if F[k - 1][j] + W[i][j] < F[k][i]:
                    F[k][i] = F[k - 1][j] + W[i][j]
                    V[k][i] = j+1
    for i in range(max(len(F), len(V))-1, 0, -1):
        if F[i] == F[i-1]:
            F.pop(i)
        if V[i] == V[i-1]:
            V.pop(i)
    V.pop()
    P = [end+1]
    for i in range(len(V)):
        if V[i][end] != math.inf:
            x = i
            break
    while end != start+1:
        end = V[x][P[-1]-1]
        x -= 1
        P.append(end)
    if len(P) == 1:
        P.append(start+1)
    return P[::-1]

In [ ]:
start_time = time.time()
G = make_graph(read_file("adjacency_matrix.txt"), 1)
res = bellman_ford(G, 1, 6)
print("Кратчайший путь:", res)
print("time: ","%s seconds" % (time.time() - start_time))

new_graph(G, res).show("basic.html")

#### Алгоритм Джонсона

In [ ]:
def bellman_ford_len(W, start, end):
    N = len(W)
    F = [[math.inf] * N for i in range(N)]
    F[0][start-1] = 0 
    for k in range(1, N): 
        for i in range(N): 
            F[k][i] = F[k - 1][i]
            for j in range(N):
                if F[k - 1][j] + W[i][j] < F[k][i]:
                    F[k][i] = F[k - 1][j] + W[i][j]
    return F[-1][end-1]

def johnson(G, start, end):
    D = make_adjacency_matrix(G)
    for i in range(len(D)):
            for j in range(len(D[i])):
                if D[i][j] == 0 and i != j:
                    D[i][j] = math.inf
    negatives = 0
    for i in range(len(D)):
        for j in range(len(D[i])):
            if D[i][j] < 0:
                negatives = 1
    if negatives != 0:
        D.append([math.inf for i in range(len(D))])
        for i in range(len(D)):
            D[i].append(0)
        nodes = []
        for i in range(len(G.nodes)):
            nodes.append(list(G.nodes)[i]['id'])
        nodes.sort()
        lens = []
        for i in range(len(nodes)):
            lens.append(bellman_ford_len(D, len(D), nodes[i]))
        for i in range(len(D)):
            D[i].pop()
        D.pop()
        for i in range(len(D)):
            for j in range(len(D[i])):
                if D[i][j] != 0 and D[i][j] != math.inf:
                    D[i][j] = D[i][j] + lens[j] - lens[i]
        for i in range(len(D)):
            for j in range(len(D[i])):
                if D[i][j] != math.inf and i != j:
                    D[i][j] += 1
    G = make_graph(D, 1)
    res = dijkstra(G, start, end)
    return res

In [ ]:
start_time = time.time()
G = make_graph(read_file("adjacency_matrix_neg.txt"), 1)
res = johnson(G, 1, 6)
print("Кратчайший путь:", res)
print("time: ","%s seconds" % (time.time() - start_time))

new_graph(G, res).show("basic.html")

#### Алгоритм Левита

In [ ]:
def levit(G, start, end):
    start -= 1
    end -= 1
    N = len(G.nodes)
    D = [math.inf for i in range(N)]
    D[start] = 0
    cache = deque([start])
    state = [2 for i in range(N)]
    state[start] = 1
    P = [-1 for i in range(N)]
    while cache:
        vertex = cache.popleft()
        state[vertex] = 0
        for i in range(len(G.neighbors(vertex+1))):
            neigh = list(G.neighbors(vertex+1))[i]-1
            for j in range(len(G.edges)):
                if list(G.edges)[j]['from'] == vertex+1 and list(G.edges)[j]['to'] == neigh+1:
                    weight = G.edges[j]['weight']
            if D[vertex] + weight < D[neigh]:
                D[neigh] = D[vertex] + weight
                if state[neigh] == 2:
                    cache.append(neigh)
                if state[neigh] == 0:
                    cache.appendleft(neigh)
                P[neigh] = vertex
                state[neigh] = 1
    path = []
    vertex = end
    while vertex != -1:
        path.append(vertex+1)
        vertex = P[vertex]
    return path[::-1]

In [ ]:
start_time = time.time()
G = make_graph(read_file("adjacency_matrix.txt"), 1)
res = levit(G, 1, 6)
print("Кратчайший путь:", res)
print("time: ","%s seconds" % (time.time() - start_time))

new_graph(G, res).show("basic.html")

#### Алгоритм Йена

In [ ]:
def yen(G, start, end, total_paths = 3):
    start -= 1
    end -= 1
    paths = [[] for i in range(total_paths)]
    paths[0] = dijkstra(G, start+1, end+1)
    for i in range(1,len(paths)):
        edges = []
        for j in range(len(G.edges)):
            edges.append([list(G.edges)[j]['from'], list(G.edges)[j]['to']])
        length = 0
        for j in range(len(G.edges)):
            if list(G.edges)[j]['from'] == paths[i-1][0] and list(G.edges)[j]['to'] == paths[i-1][1]:
                length = list(G.edges)[j]['weight']
        G = remove_edge(G, [paths[i-1][0], paths[i-1][1]])
        paths[i] = dijkstra(G, start+1, end+1)
        G.add_edge(paths[i-1][0], paths[i-1][1], weight=length)
    return paths

In [ ]:
start_time = time.time()
G = make_graph(read_file("adjacency_matrix_yen.txt"), 1)
res = yen(G, 1, 6)
print("Кратчайший путь:", res)
print("time: ","%s seconds" % (time.time() - start_time))

new_graph(G, res[1]).show("basic.html")